# Projet 4

In [132]:
import requests
from bs4 import BeautifulSoup
import re
import multiprocessing as mp

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [156]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    content= requests.get(url).content
    soup = BeautifulSoup(content)
    dico={}
    list_plans=[]
    list_stock=[]
    list_price=[]
    list_db=[]

    for element in soup.find_all('ul',{'class' : "pricing-table-list"}):
        list_stock.append(element.find('li').find_next_sibling().find_next_sibling().find_next_sibling().text.split()[0])
        list_db.append(element.find('li').find_next_sibling().find_next_sibling().find_next_sibling().find_next_sibling().text.split()[0])

    for element in soup.find_all('h2'):
        list_plans.append(element.text)

    for element in soup.find_all('span',{'class' : "pricing-table-price"}):
        list_price.append(element.text.split()[0])

    for i in range(len(list_plans)):
        dico[list_plans[i]] = {'price':list_price[i],'storage':list_stock[i],'databases':int(list_db[i])}

    return dico

In [157]:
get_prices_from_url(URL_PAGE2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [170]:
def extract_beer_infos(url):
    
    soup = BeautifulSoup(requests.get(url).content)
    infos = {
        'name': soup.find("h1").text,
        'note': int(soup.find('div',{'class':"stars"}).get("data-percent")),
        'price': float(soup.find('span',{'class':"price"}).text.split(" ")[0].replace(',','.')),
        'volume': int(soup.find('dd',{'class':"small-6 medium-9 columns js-beer-volume"}).get("data-volume").split(" ")[0]),
    }
    return infos

In [171]:
extract_beer_infos("https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33")

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [145]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [144]:
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    beer_list = requests.get(url).json()
    for i in range(len(beer_list['items'])):
        beer_pages.append(beer_list['items'][i]['contentReference'])
    # Sequential version (slow):
    beers = []
    url_site = 'https://www.beerwulf.com'
    for j in range(len(beer_list)):
        beers.append(extract_beer_infos(url_site + beer_pages[j]))
    # Parallel version (faster):
    beers = []
    pool = mp.Pool()
    for j in range(len(beer_pages)):
        pool.apply_async(beers.append(extract_beer_infos(url_site + beer_pages[j])))
    pool.close()
    pool.join()
    return beers

In [173]:
extract_beer_list_infos(URL_BEERLIST_FRANCE)

[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay',
  'note': 60,
  'price': 38.99,
  'volume': 25},
 {'name': 'Brasserie De Sutter Brin de Folie',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'La Cristal IPA du Mont Blanc',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
 {'name': 'La Parisienne Le Titi Parisien',
  'note': 70,
  'price': 38.99,
  'volume': 33},
 {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
 {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
 {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
 {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
 {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]

In [154]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'price': '$99', 'storage': '1TB', 'databases': 100}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [172]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.037s

OK
